# Autocorrelation Calculation
Calculate the autocorrelation from lag 1 through 6.

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
# load expression df
expression_df = pd.read_csv("gene_expression_original.csv", sep='\t', index_col=0)

In [3]:
expression_df.head()

,2MOa,2MOb,2MOc,2MOd,4MOa,4MOb,4MOc,6MOa,6MOb,6MOc,...,18MOb,18MOc,21MOa,21MOb,21MOc,23MOa,23MOb,23MOc,23MOd,Gene
SSMa000210,0.888550,0.308417,0.510500,0.531384,0.083351,0.234203,0.174035,0.284850,0.430807,0.498110,...,0.424678,0.436213,0.472740,0.267945,0.427943,0.277950,0.623046,1.051232,0.522695,SSMa000210
SSMa000220,15.830669,14.932662,17.866173,17.213263,13.277656,15.281277,11.521445,8.690235,9.344659,8.101292,...,9.257452,11.476075,9.296245,9.079901,9.547774,10.163612,10.225794,11.828746,8.392287,SSMa000220
SSMa000230,44.066716,46.721348,44.645520,47.703918,41.211085,40.637081,30.891665,21.318561,15.956269,19.432475,...,26.864437,32.618656,21.580500,19.431561,24.724316,23.167285,23.018588,33.909775,18.480654,SSMa000230
SSMa000260,2.847206,2.179543,3.558553,3.159124,1.386708,2.267895,1.548236,1.382103,1.594781,1.521632,...,1.668198,1.784632,1.484056,1.407730,1.537458,2.031528,1.608598,1.861561,1.780423,SSMa000260
SSMa000270,35.672820,35.669393,36.606042,37.090886,36.280303,37.302112,25.348904,19.089341,15.674094,16.104046,...,20.898470,25.602797,18.189956,24.465830,29.391152,22.650435,21.429731,32.440270,19.669494,SSMa000270


In [44]:
expression_df_long = pd.melt(expression_df, id_vars="Gene", var_name="Time", value_name="TPM")

In [45]:
expression_df_long["Log_TPM"] = np.log10(expression_df_long["TPM"])

In [46]:
import re
def extract_number(mystring):
    numbers = re.findall("^\d+", mystring)
    return int(numbers[0])

In [47]:
expression_df_long["Month"] = expression_df_long["Time"].apply(extract_number)

In [48]:
expression_df_long.sort_values(["Gene", "Month"], inplace=True)

## Hypothesis testing of Autocorrelation
Note that there are multiple measurements at each timepoint. I repeatedly calculate autocorrelation at lag 1 through 6 by sampling one measurement from each timepoint. If p-value is smaller than 0.05, I consider the autocorrelation significantly different from zero.

In [50]:
from statsmodels.tsa.stattools import acf

In [51]:
gene_names = list(expression_df.index)

In [53]:
first_gene = gene_names[0]

In [54]:
selected_gene_expression = expression_df_long.loc[expression_df_long["Gene"] == first_gene, :]

In [55]:
acf_mat = np.zeros((100, 9))
for j in range(100):
    sample_selected_gene_expression = selected_gene_expression.groupby("Month").sample(n=1)
    sample_acfs = acf(sample_selected_gene_expression["Log_TPM"], nlags=9)
    acf_mat[j, :] = sample_acfs[1:10]

In [56]:
mean_acf_allgenes = pd.DataFrame(np.zeros((len(gene_names), 9)))
mean_acf_allgenes.index = gene_names
sd_acf_allgenes = pd.DataFrame(np.zeros((len(gene_names), 9)))
sd_acf_allgenes.index = gene_names
lb_acf_allgenes = pd.DataFrame(np.zeros((len(gene_names), 9)))
lb_acf_allgenes.index = gene_names
ub_acf_allgenes = pd.DataFrame(np.zeros((len(gene_names), 9)))
ub_acf_allgenes.index = gene_names
pval_acf_allgenes = pd.DataFrame(np.zeros((len(gene_names), 9)))
pval_acf_allgenes.index = gene_names

In [58]:
for i in range(len(gene_names)):
    if (i % 100 == 0):
        print(f"Processing gene {i}")
    gname = gene_names[i]
    selected_gene_expression = expression_df_long.loc[expression_df_long["Gene"] == gname, :]
    acf_mat = np.zeros((100, 9))
    for j in range(100):
        sample_selected_gene_expression = selected_gene_expression.groupby("Month").sample(n=1)
        sample_acfs = acf(sample_selected_gene_expression["Log_TPM"], nlags=9)
        acf_mat[j, :] = sample_acfs[1:10]
    prob_gt0 = np.mean(acf_mat > 0, axis=0)
    prob_lt0 = np.mean(acf_mat < 0, axis=0)
    pval_acf_allgenes.loc[gname, :] = np.minimum(prob_gt0, prob_lt0) * 2
    mean_acf_allgenes.loc[gname, :] = np.mean(acf_mat, axis=0)
    sd_acf_allgenes.loc[gname, :] = np.std(acf_mat, axis=0)
    lb_acf_allgenes.loc[gname, :] = np.quantile(acf_mat, q=0.025, axis=0)
    ub_acf_allgenes.loc[gname, :] = np.quantile(acf_mat, q=0.975, axis=0)


Processing gene 0
Processing gene 100
Processing gene 200
Processing gene 300
Processing gene 400
Processing gene 500
Processing gene 600
Processing gene 700
Processing gene 800
Processing gene 900
Processing gene 1000
Processing gene 1100
Processing gene 1200
Processing gene 1300
Processing gene 1400
Processing gene 1500
Processing gene 1600
Processing gene 1700
Processing gene 1800
Processing gene 1900
Processing gene 2000
Processing gene 2100
Processing gene 2200
Processing gene 2300
Processing gene 2400
Processing gene 2500
Processing gene 2600
Processing gene 2700
Processing gene 2800
Processing gene 2900
Processing gene 3000
Processing gene 3100
Processing gene 3200
Processing gene 3300
Processing gene 3400
Processing gene 3500
Processing gene 3600
Processing gene 3700
Processing gene 3800
Processing gene 3900
Processing gene 4000
Processing gene 4100
Processing gene 4200
Processing gene 4300
Processing gene 4400
Processing gene 4500
Processing gene 4600
Processing gene 4700
Proc

In [60]:
pval_acf_allgenes.head(10)

,0,1,2,3,4,5,6,7,8
SSMa000210,0.72,1.00,0.70,0.26,1.00,0.74,0.16,0.98,0.86
SSMa000220,0.06,0.00,0.16,0.06,0.32,0.20,0.08,0.66,0.40
SSMa000230,0.52,0.30,0.00,0.56,0.24,0.52,0.10,0.44,0.92
SSMa000260,0.68,0.84,0.04,0.44,0.04,0.76,0.64,0.98,0.84
SSMa000270,0.96,0.42,0.24,0.84,0.30,0.74,0.44,0.98,0.96
SSMa000280,0.74,0.22,0.26,0.62,0.94,0.76,0.68,0.68,0.94
SSMa000330,0.90,0.36,0.78,0.32,0.74,0.98,0.90,0.30,0.84
SSMa000340,0.70,0.12,0.66,0.96,0.98,0.72,0.74,0.82,0.74
SSMa000350,0.68,0.14,0.74,0.68,0.88,0.90,0.54,0.58,0.44
SSMa000370,0.00,0.00,0.14,0.00,0.10,0.40,0.02,0.42,0.42


In [61]:
pval_acf_allgenes.columns = [f'Pval_{i}' for i in np.arange(1, 10)]
mean_acf_allgenes.columns = [f"Mean_{i}" for i in np.arange(1, 10)]
sd_acf_allgenes.columns = [f"SD_{i}" for i in np.arange(1, 10)]
lb_acf_allgenes.columns = [f"LB_{i}" for i in np.arange(1, 10)]
ub_acf_allgenes.columns = [f"UB_{i}" for i in np.arange(1, 10)]


In [62]:
acf_full_results = pd.concat([pval_acf_allgenes, mean_acf_allgenes, sd_acf_allgenes, lb_acf_allgenes, ub_acf_allgenes], axis=1)

In [63]:
acf_full_results.head()

,Pval_1,Pval_2,Pval_3,Pval_4,Pval_5,Pval_6,Pval_7,Pval_8,Pval_9,Mean_1,...,LB_9,UB_1,UB_2,UB_3,UB_4,UB_5,UB_6,UB_7,UB_8,UB_9
SSMa000210,0.72,1.00,0.70,0.26,1.00,0.74,0.16,0.98,0.86,-0.065487,...,-0.170298,0.351926,0.252192,0.389373,0.174376,0.336367,0.173388,0.079048,0.239542,0.210010
SSMa000220,0.06,0.00,0.16,0.06,0.32,0.20,0.08,0.66,0.40,0.239587,...,-0.374807,0.436202,-0.062070,0.105105,0.005555,0.046148,0.320359,0.419510,0.161087,0.103147
SSMa000230,0.52,0.30,0.00,0.56,0.24,0.52,0.10,0.44,0.92,0.091750,...,-0.227515,0.302227,0.088562,-0.046056,0.196136,0.034833,0.191245,0.307431,0.292873,0.181350
SSMa000260,0.68,0.84,0.04,0.44,0.04,0.76,0.64,0.98,0.84,0.119879,...,-0.337034,0.467024,0.161834,-0.026755,0.267943,-0.020589,0.184505,0.271071,0.182817,0.139403
SSMa000270,0.96,0.42,0.24,0.84,0.30,0.74,0.44,0.98,0.96,0.012639,...,-0.324155,0.377218,0.226754,0.167120,0.314854,0.155841,0.274730,0.408502,0.276437,0.202365


In [64]:
acf_full_results.to_csv("acf_full_results.tsv", sep='\t')